In [38]:
import json
from bs4 import BeautifulSoup
import urllib.request

In [39]:
f = open('url_modified.json', encoding = 'utf-8')
data = json.load(f)
f.close()

In [5]:
data2= data

In [6]:
data3 = data

In [7]:
data3['2000']['1']['age'] = '예시'

In [8]:
data3['2000']['1']['age']

'예시'

In [9]:
data['2000']['1']['url']

'https://movie.naver.com/movie/bi/mi/basic.naver?code=15899'

In [22]:
data['2000']['1']

{'title': '공동경비구역 JSA',
 'year': 2000,
 'audience': 2447133,
 'date': '2000-09-08 00:00:00',
 'url': 'https://movie.naver.com/movie/bi/mi/basic.naver?code=15899'}

### 최종 코드

In [40]:
data_final = data

In [41]:
def getInfoSpec(html):
    dtList = html.find_all('dt')
    ddList = html.find_all('dd')
 
    # 딕셔너리로 가져올 정보를 미리 정해 놓는다.
    info = {'장르':'',
            '상영시간':'',
            '국가':'',
            '개봉':'',
            '감독':'',
            '출연':'',
            '등급':''}
 
    # dt, dd의 개수만큽
    for dt, dd in zip(dtList, ddList):
        if '개요' in dt.text:
            for span in dd.find_all('span'):
                if '분' in span.text:
                    info['상영시간'] = span.text
                else:
                    for a in span.find_all('a'):
                        if 'genre' in str(a):
                            # 공연실황 제외 - 이건 뭐 하는건지?
                            # 멜로/로맨스 - 이상한 19금 영화가 엄청 많음, 어떻게 거를 방법이...
                            if '공연실황' in a.text:
                                continue
                            if info['장르'] != '':
                                info['장르'] += ', ' + a.text
                            else:
                                info['장르'] = a.text
                        # 국가도 여러개인 경우가 있네요
                        elif 'nation' in str(a):
                            if info['국가'] != '':
                                info['국가'] += ', ' + a.text
                            else:
                                info['국가'] = a.text
                        # 재개봉 영화의 경우 개봉이 여러개로 들어온다.
                        elif 'open' in str(a):
                            if info['개봉'] != '':
                                info['개봉'] += a.text
                            else:
                                info['개봉'] = a.text
        elif '감독' in dt.text:
            for a in dd.find_all('a'):
                # 감독이 여러명인 경우가 있어서 - 루소 형제 등?
                if info['감독'] != '':
                    info['감독'] += ', ' + a.text
                else:
                    info['감독'] = a.text
        elif '출연' in dt.text:
            for a in dd.find_all('a'):
                # 메인 출연진이 아니라면 pass
                if '더보기' in a.text:
                    continue
                if info['출연'] != '':
                    info['출연'] += ', ' + a.text
                else:
                    info['출연'] = a.text
        # 등급의 경우 한국 등급만 가져오려면 수정이 필요
        # 한국 등급이 없는 경우를 대비해서 일본, 미국 등의 등급이라도 가져오기 위해
        # 별다른 수정은 안 했습니다.
        elif '등급' in dt.text:
            for a in dd.find_all('a'):
                if 'grade' in str(a):
                    if info['등급'] != '':
                        info['등급'] += ', ' + a.text
                    else:
                        info['등급'] = a.text
    return info

In [42]:
def getSummary(html):
    
    # 줄거리(시놉시스) 수집
    summary_list = []
    try:
        contents_head = soup.find('h5',class_='h_tx_story').text
        content_infos.append(contents_head)
    except:
        None

    contents_texts = soup.select('div.story_area > p.con_tx')
    
    if len(contents_texts) == 0:
        summary_list.append("줄거리 오류")
    else:
        for contents in contents_texts:
            temp = contents.text
            temp = temp.replace('\r', '')
            temp = temp.replace('\xa0', '') 
            summary_list.append(temp)
            
    return summary_list

def getNetScore(html):
    try:
        net_score_temp = soup.find("div",class_="score score_left")
        net_score = net_score_temp.text[-6:]
        try:
            net_score = net_score.replace('\n','')
        except:
            None
        return net_score
    except:
        return None

In [43]:
for year in range(1980,2023):
    for rank in range(1,101):
        try:
            url = data_final[str(year)][str(rank)]['url']
            with urllib.request.urlopen(url) as url:
                doc = url.read()
                soup = BeautifulSoup(doc, 'html.parser')
                info = getInfoSpec(soup)
                summary = getSummary(soup)
                net_score = getNetScore(soup)
                data_final[str(year)][str(rank)]['genre'] = info['장르']
                data_final[str(year)][str(rank)]['actor'] = info['출연']
                data_final[str(year)][str(rank)]['country'] = info['국가']
                data_final[str(year)][str(rank)]['director'] = info['감독']
                data_final[str(year)][str(rank)]['age'] = info['등급']
                data_final[str(year)][str(rank)]['net_score'] = net_score
                data_final[str(year)][str(rank)]['synopsis'] = summary
        except:
            print(str(year),str(rank))
            print(data_final[str(year)][str(rank)])
    print('{}년 작업 완료'.format(year))

1980년 작업 완료
1981년 작업 완료
1982 72
{'title': '화니 피플', 'year': 1982, 'audience': 24660, 'date': '1982-05-20 00:00:00', 'url': None}
1982년 작업 완료
1983 45
{'title': '완전한 결혼', 'year': 1982, 'audience': 39948, 'date': '1982-12-31 00:00:00', 'url': None}
1983년 작업 완료
1984년 작업 완료
1985년 작업 완료
1986년 작업 완료
1987년 작업 완료
1988 46
{'title': '7 소복', 'year': 1988, 'audience': 75946, 'date': '1988-01-01 00:00:00', 'url': None}
1988년 작업 완료
1989 58
{'title': '유환도사', 'year': 1989, 'audience': 65013, 'date': '1989-01-07 00:00:00', 'url': None}
1989년 작업 완료
1990년 작업 완료
1991년 작업 완료
1992년 작업 완료
1993년 작업 완료
1994년 작업 완료
1995년 작업 완료
1996년 작업 완료
1997년 작업 완료
1998년 작업 완료
1999년 작업 완료
2000년 작업 완료
2001년 작업 완료
2002년 작업 완료
2003년 작업 완료
2004년 작업 완료
2005년 작업 완료
2006년 작업 완료
2007년 작업 완료
2008년 작업 완료
2009년 작업 완료
2010년 작업 완료
2011년 작업 완료
2012년 작업 완료
2013년 작업 완료
2014년 작업 완료
2015년 작업 완료
2016년 작업 완료
2017년 작업 완료
2018년 작업 완료
2019년 작업 완료
2020년 작업 완료
2021년 작업 완료
2022년 작업 완료


In [44]:
# 확인 코드
data_final['1980']['30']

{'title': '평양맨발',
 'year': 1980,
 'audience': 123376,
 'date': '1980-05-17 00:00:00',
 'url': 'https://movie.naver.com/movie/bi/mi/basic.naver?code=23455',
 'genre': '',
 'actor': '이주일, 마영달, 곽은경, 이주일, 마영달, 곽은경',
 'country': '대한민국',
 'director': '남기남, 남기남, 신고',
 'age': '12세 관람가',
 'net_score': '8.80',
 'synopsis': ['황석두는 맨발로 달리기를 잘했다. 일본은 우리나라 포목상들에게 중국과의 거래를 근절시키기 위해 새 무역법을 제정했다. 펑양굴지의 포목상 주인 김만호는 무역법 제정으로 큰타격을 받는다. 김만호는 고민끝에 맨발을 만나 사정을 얘기하고 포목 운반을 부탁한다. 맨발은 역전일대의 왜경들의 눈을 피해 포목을 운반한다. 맨발은 고등소학교에 입학하나 유도부장 마사요시로 인해 퇴학당한후 결혼한다. 맨발은 초야도 치루지 못하고 괴롭혀 오는 마사요시를 때려 눕힌 죄로 만주로 피한다. 맨발은 그곳에서 곤경에 빠져있던 여옥을 구해 주나 아기를 남겨놓은채 달아난다. 맨발은 애기 엄마를 찾기위해 상해와 만주를 헤맨다. 맨발은 탄광촌에서 노무자를 괴롭히는 일인 조합의 횡포를 몰아내기도 한다. 다시 애기 엄마를 찾던 맨발은 일본인 거부의 애첩이 되어 마차에서 내리는 애기엄마를 보고 놀란다.']}

In [45]:
data_final['2007']['45']

{'title': '조폭마누라3',
 'year': 2006,
 'audience': 1039848,
 'date': '2006-12-28 00:00:00',
 'url': 'https://movie.naver.com/movie/bi/mi/basic.naver?code=39756',
 'genre': '코미디, 액션',
 'actor': '서기(임아령), 이범수(한기철), 서기, 이범수',
 'country': '대한민국',
 'director': '조진규, 조진규',
 'age': '15세 관람가',
 'net_score': '7.53',
 'synopsis': ['홍콩 최고의 명문 조직 화백련 보스의 외동딸 아령(서기). 보스 임회장은 조직간 세력다툼이 벌어지자 그녀를 한국으로 피신시키고 한국의 동방파 보스 양사장에게 딸의 안전을 부탁한다. 양사장은 밀수로나마 중국어 실력을 쌓아온 No.3 기철(이범수)을 믿고(?) 아령의 보호를 맡긴다. 아령의 실체를 전혀 모른 채 관광 가이드쯤으로 생각한 기철과 꽁치(오지호), 도미(조희봉)는 명문가의 후계자다운 아령의 도도함 때문에 당황스러울 뿐. 급기야 밀수용 중국어 실력에 한계를 느낀 기철은 연변처녀 연희(현영)를 급초빙하게 된다.기철과 아령 사이에서 살벌한 기운이 감도는 가운데 연희는 생존본능적인 엽기 통역을 구사하며 이들의 좌충우돌 동거에 합류한다. 그러나 아령의 목숨을 노린 킬러가 홍콩에서 한국으로 급파되면서 아령 뿐 아니라 기철 일당마저 생명의 위협을 받게 되는데…']}

In [46]:
data_trial['2022']['3']

{'title': '한산: 용의 출현',
 'year': 2022,
 'audience': 7262885,
 'date': '2022-07-27 00:00:00',
 'url': 'https://movie.naver.com/movie/bi/mi/basic.naver?code=194196',
 'genre': '액션, 드라마',
 'actor': '박해일(이순신), 변요한(와키자카 야스하루), 안성기(어영담), 박해일, 변요한, 안성기',
 'country': '대한민국',
 'director': '김한민, 김한민',
 'age': '12세 관람가',
 'net_score': '8.75\n\n',
 'synopsis': ['1592년 4월, 조선은 임진왜란 발발 후 단 15일 만에 왜군에 한양을 빼앗기며 절체절명의 위기에 놓인다.조선을 단숨에 점령한 왜군은 명나라로 향하는 야망을 꿈꾸며 대규모 병역을 부산포로 집결시킨다.한편, 이순신 장군은 연이은 전쟁의 패배와 선조마저 의주로 파천하며 수세에 몰린 상황에서도조선을 구하기 위해 전술을 고민하며 출전을 준비한다.하지만 앞선 전투에서 손상을 입은 거북선의 출정이 어려워지고,거북선의 도면마저 왜군의 첩보에 의해 도난 당하게 되는데…왜군은 연승에 힘입어 그 우세로 한산도 앞바다로 향하고,이순신 장군은 조선의 운명을 가를 전투를 위해 필사의 전략을 준비한다.1592년 여름, 음력 7월 8일 한산도 앞바다,압도적인 승리가 필요한 조선의 운명을 건 지상 최고의 해전이 펼쳐진다.']}

In [47]:
# .json 파일로 저장하기

with open('total_movie_spec.json', 'w',encoding = "utf-8") as make_file:
  json.dump(data, make_file, indent=4, ensure_ascii=False)